# CubeSatLab/Design I 
Assignment 1
 
## Analyzing CubeSat Missions
Objective: To analyze and visualize CubeSat mission data from the provided Wikipedia source.


## Instructions

### Step 1: Data Gathering
- Visit the Wikipedia page on CubeSats: [List of CubeSats](https://en.wikipedia.org/wiki/List_of_CubeSats)
- Collect relevant data from the CubeSat missions, such as launch year, size, mission status (success or failure), mission category, and deployment type (ISS or direct launch).

### Step 2: Data Analysis
Organize the data and calculate the required statistics for the following aspects:
- Distribution (percentage) of CubeSat sizes.
- Number of CubeSat launches per year.
- Number of mission failures and successes over the years (distinguish between launch failure and mission failure).
- Distribution (percentage) of missions based on the mission category.
- Distribution (percentage) of deployment types (launch from ISS or direct launch).

### Step 3: Data Visualization

Using a tool like Microsoft Excel, Google Sheets, or any other data visualization software, create appropriate graphs for each of the analyzed aspects.
For each analysis, include clear and labeled graphs such as pie charts, bar graphs, and line charts to represent the data effectively.
Include a title, labels, and legends to make the graphs informative.

### Step 4: Interpretation and Conclusion

Write a brief interpretation of the results for each analysis.
Discuss any trends, patterns, or insights you’ve gained from the data.
Conclude the assignment by summarizing the most important findings.

### Grading Rubric:

- Data Gathering: 10 points
- Data Analysis: 20 points
- Data Visualization: 20 points
- Interpretation and Conclusion: 20 points
- Presentation and Clarity: 10 points
- Adherence to Instructions: 10 points
- Overall Quality: 10 points

# Data Gathering

In [131]:
import pandas as pd
import numpy as np
import html5lib
import re

In [132]:
with open("Listofcubesats.html", "r", encoding="utf-8") as f:
	html_text = f.read()

tables = pd.read_html(html_text)

table = tables[0]

table.head(20)

/tmp/ipykernel_163972/3906797611.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html_text)


,Name,COSPAR ID (NORAD ID),Type,Organisation,Mission,Mission status,Launch date (UTC),Launch vehicle,Reentry date,Remarks
0,1KUNS-PF,1998-067NQ (43467),1U,University of Nairobi,"Mapping of Kenya's land mass, monitoring of th...",Active,2 Apr 2018,Falcon-9 SpaceX CRS-14,NaN,Deployed from ISS in May 2018.
1,Aalto-1,2017-036L (42775),3U,Aalto University and Finnish Meteorological In...,Technical demonstration of a miniaturized spec...,Active,23 Jun 2017,PSLV-C38 from India,NaN,Second Finnish satellite[168]
2,Aalto-2,1998-067MJ (42729),2U,"Aalto University, Finland",Atmospheric research,Inactive,18 Apr 2017,Atlas V,NaN,First Finnish satellite. Part of the QB50 proj...
3,AAU CubeSat[46],2003-031G (27846),1U,Aalborg University,Technology [47],Completed,30 Jun 2003[48],Rokot / Briz-KM,NaN,"Battery problems, deactivated on 22 September ..."
4,AAUSat-2[72],2008-021F (32788),1U,"Aalborg University, Denmark",ADCS system and a gamma ray detector,Completed,28 Apr 2008,PSLV-CA,NaN,NaN
5,AAUSAT3[113],2013-009B (39087),1U,"Aalborg University, Denmark",Double AIS system for tracking ships in Arctic...,Active,25 Feb 2013[76],PSLV-CA C20,NaN,Denmark's CubeSat number 4
6,ACRUX-1,2019-037E (44369),1U,Melbourne Space Program[190],Technical demonstration,Success,29 Jun 2019,Electron,NaN,"Second Australian student-built satellite, fol..."
7,AeroCube 8A,NaN,1.5U,Aerospace Corp.,NaN,Active,20 May 2015,Atlas V,NaN,[156]
8,AeroCube 8B,NaN,1.5U,Aerospace Corp.,NaN,Active,20 May 2015,Atlas V,NaN,[156]
9,AeroCube-1,NaN,1U,The Aerospace Corporation,Technology demonstration,Destroyed. Launch failure[65],26 Jul 2006[66],Dnepr,NaN,NaN


In [133]:
# Clean up data (remove references)
def remove_numeric_brackets(val):
    if pd.isna(val):
        return val
    if not isinstance(val, str):
        return val
    cleaned = re.sub(r'\s*\[\d+\]', '', val)
    return cleaned.strip()

for col in table.columns:
    if pd.api.types.is_categorical_dtype(table[col]):
        new_cats = [remove_numeric_brackets(c) if isinstance(c, str) else c for c in table[col].cat.categories]
        table[col] = table[col].cat.rename_categories(new_cats)
    elif table[col].dtype == object:
        table[col] = table[col].apply(remove_numeric_brackets)


/tmp/ipykernel_163972/4038235945.py:11: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(table[col]):


In [134]:
# normalize values in the 'Type' column and convert to categorical dtype
def normalize_type(t):
	if pd.isna(t):
		return np.nan
	s = str(t).strip()
	s_lower = s.lower()

	# canonicalize common CubeSat size variants
	if re.search(r'\b2u\b', s_lower):
		return '2U'
	if re.search(r'\b3u\b', s_lower):
		return '3U'
	if re.search(r'\b6u\b', s_lower):
		return '6U'
	if re.search(r'\b1u\b', s_lower):
		return '1U'
	# keep explicit femto label
	if 'femto' in s_lower:
		return 'Other'
	# map non-standard launcher text to Other
	if 'in-orbit cubesat' in s_lower or ('launcher' in s_lower and 'cubesat' in s_lower):
		return 'Other'
	# otherwise return cleaned original string
	return s

table['Type'] = table['Type'].apply(normalize_type).astype('category')

# verify conversion and inspect categories
print("n_categories:", table['Type'].nunique(dropna=True))
print(table['Type'].value_counts(dropna=False))

n_categories: 7
Type
1U       123
3U        64
2U        23
6U        20
1.5U      12
12U        3
NaN        3
Other      2
Name: count, dtype: int64


In [135]:
# convert the 'Mission status' column to categorical dtype
table['Mission status'] = table['Mission status'].astype('category')

# verify conversion and inspect categories
print("n_categories:", table['Mission status'].nunique(dropna=True))
print(table['Mission status'].value_counts(dropna=False))

n_categories: 31
Mission status
Active                                                                                             80
Launched                                                                                           50
NaN                                                                                                34
Completed                                                                                          19
Destroyed. Launch failure                                                                          18
Failed                                                                                              8
Successful                                                                                          6
Complete (Re-entered)                                                                               5
Complete                                                                                            4
Operational                                       

In [136]:
def map_mission_status_to_outcome(status):
    if pd.isna(status):
        return 'Unknown'
    s = str(status).lower()
    # failure indicators
    if any(token in s for token in ['fail', 'failure', 'failed', 'lost', 'lost contact', 'destroy', 'anomaly', 'no signal', 'burnup', 'canceled']):
        return 'Failure'
    # success indicators
    if any(token in s for token in ['active', 'complete', 'completed', 'success', 'succeeded', 'operational', 'deployed', 'in orbit', 'on-orbit', 'successful', 'commissioned', 'returned']):
        return 'Success'
    # fallback for ambiguous/other statuses
    return 'Unknown'

table['mission_outcome'] = table['Mission status'].apply(map_mission_status_to_outcome).astype('category')

# quick check
print(table['mission_outcome'].value_counts(dropna=False))

mission_outcome
Success    124
Unknown     58
Failure     34
NaN         34
Name: count, dtype: int64


In [137]:
# Map detailed Mission text to a small set of high-level categories
def map_mission_to_category(m):
    if pd.isna(m):
        return 'Unknown'
    s = str(m).lower()
    # keyword-driven coarse categories (order matters: more specific first)
    mapping = [
        ('Earth Observation', ['earth', 'mapping', 'imaging', 'remote sensing', 'imagery', 'land', 'ocean', 'terrain', 'observation', 'camera']),
        ('Space Weather', ['space weather', 'space-weather', 'spaceweather', 'space weather']),
        ('Communication', ['communic', 'comms', 'amateur', 'ham', 'relay', 'telemetry']),
        ('Education', ['student', 'university', 'educat', 'student-built', 'education', 'outreach']),
        ('Science', ['science', 'scientif', 'research', 'experiment', 'measure']),
        ('Technology', ['technology', 'tech', 'demonstrat', 'prototype', 'test', 'technology demonstration', 'tech demo']),
    ]
    for cat, keywords in mapping:
        if any(kw in s for kw in keywords):
            return cat
    # fallback for short/ambiguous descriptions
    return 'Other'

# Apply mapping and convert to categorical dtype
table['mission_category'] = table['Mission'].apply(map_mission_to_category).astype('category')

# Merge very small categories into 'Other' to further reduce category count
counts = table['mission_category'].value_counts()
small_cats = counts[counts < 5].index.difference(['Unknown', 'Other'])
if len(small_cats):
    table.loc[table['mission_category'].isin(small_cats), 'mission_category'] = 'Other'
    table['mission_category'] = table['mission_category'].astype('category')

# Quick inspection
print(table['mission_category'].value_counts(dropna=False))

mission_category
Technology           89
Unknown              42
Other                29
Science              28
Earth Observation    26
Communication        20
Education             9
Space Weather         7
Name: count, dtype: int64


In [138]:
# create a new column indicating whether the satellite was deployed from the ISS based on the Remarks column
def detect_iss_deployment(remark):
    if pd.isna(remark):
        return 'Unknown'
    s = str(remark).lower()
    # common explicit patterns
    if re.search(r'deployed\s+from\s+(the\s+)?iss', s) or 'international space station' in s:
        return 'Yes'
    # some remarks may say "deployed" with date but not mention ISS; treat those as No
    return 'No'

table['deployed_from_ISS'] = table['Remarks'].apply(detect_iss_deployment).astype('category')

# quick check
print(table['deployed_from_ISS'].value_counts(dropna=False))

deployed_from_ISS
No         147
Unknown     76
Yes         27
Name: count, dtype: int64


In [139]:
table['Launch date (UTC)'] = pd.to_datetime(table['Launch date (UTC)'], errors='coerce').dt.normalize()

print(table['Launch date (UTC)'].dtype)
table.info()

datetime64[ns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Name                  250 non-null    object        
 1   COSPAR ID (NORAD ID)  175 non-null    object        
 2   Type                  247 non-null    category      
 3   Organisation          248 non-null    object        
 4   Mission               208 non-null    object        
 5   Mission status        216 non-null    category      
 6   Launch date (UTC)     249 non-null    datetime64[ns]
 7   Launch vehicle        250 non-null    object        
 8   Reentry date          49 non-null     object        
 9   Remarks               174 non-null    object        
 10  mission_outcome       216 non-null    category      
 11  mission_category      250 non-null    category      
 12  deployed_from_ISS     250 non-null    category      
dtypes: ca